# Projeto 2 - Ciência dos Dados

Nome: Matheus Freitas Sant'Ana

Nome: Thiago Lopes David

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [22]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [23]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [24]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [25]:
#Produto escolhido:
produto = 'uber'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [26]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

NameError: name 'auth' is not defined

Salvando os dados em uma planilha Excel:

In [27]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [28]:
import os
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/thiagodavid/Documents/Cienciadosdados/P2-Ciencia-dos-Dados


In [29]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [30]:
tweets = pd.read_excel('uber.xlsx', sheet_name = "Treinamento")
tweets_teste = pd.read_excel('uber.xlsx', sheet_name = "Teste")

In [31]:
#DataFrame_irrelevantes = tweets.loc[tweets['Classificação'] == 0]
#contagem_irrel = pd.Series(cleanup(" ".join(DataFrame_irrelevantes["Treinamento"])).split()).value_counts()
#freqr_ir = pd.Series(contagem_irrel).value_counts(True)*100
DataFrame_irrelevantes = tweets.loc[tweets['Classificação'] == 0]
texto_ir = ''.join(DataFrame_irrelevantes['Treinamento'])
texto_clean_ir = cleanup(texto_ir)
lista_ir = texto_clean_ir.split()
freq_ir = pd.Series(lista_ir).value_counts(True)*100
freq_ir

uber                               4.783599
o                                  3.449398
um                                 2.700944
de                                 2.570778
e                                  2.408070
eu                                 2.147738
aí                                 1.854865
que                                1.496909
a                                  1.366743
no                                 1.204035
tava                               1.204035
do                                 1.171494
pegar                              0.976245
com                                0.976245
hoje                               0.943703
fui                                0.911162
na                                 0.911162
3                                  0.878620
sou                                0.878620
eh                                 0.878620
ne                                 0.878620
pra                                0.878620
não                             

In [32]:
#DataFrame_relevantes = tweets.loc[tweets['Classificação'] == 1]
#contagem_rel = pd.Series(cleanup(" ".join(DataFrame_relevantes["Treinamento"])).split()).value_counts()
#contagem_rel
DataFrame_relevantes = tweets.loc[tweets['Classificação'] == 1]
texto_r = ''.join(DataFrame_relevantes['Treinamento'])
texto_clean_r = cleanup(texto_r)
lista_r = texto_clean_r.split()
freq_r = pd.Series(lista_r).value_counts(True)*100
freq_r

uber           4.834606
de             3.265479
e              2.756573
o              2.629347
que            2.205259
a              1.993215
pra            1.908397
eu             1.781170
um             1.738762
no             1.441900
q              1.357082
do             1.145038
não            1.060221
com            1.060221
é              0.890585
na             0.805768
minha          0.763359
ele            0.763359
da             0.763359
meu            0.720950
me             0.678541
uma            0.636132
https          0.636132
//t            0.636132
mais           0.551315
se             0.551315
vai            0.508906
tá             0.508906
por            0.466497
em             0.466497
                 ...   
paguei         0.042409
tbmfazendo     0.042409
homem          0.042409
transa         0.042409
pedido         0.042409
5,90           0.042409
irmã           0.042409
extramente     0.042409
saber          0.042409
outra          0.042409
estrelasagr    0

In [33]:
soma_r = len(freq_r)
soma_ir = len(freq_ir)
total = soma_r + soma_ir

#Dados Empíricos
p_relevante = soma_r/total
p_irrelevante =soma_ir/total

In [34]:
def classificador_naive_bayes(frase):
    
    frase_clean = cleanup(frase)
    lista_frase = frase_clean.split()
    
    #P(Frase|Relevante)
    p_frase_relevante = freq_r.loc[lista_frase].prod()
    
    #P(Frase|Irrelevante)
    p_frase_irrelevante = freq_ir.loc[lista_frase].prod()
    
    #P(Relevante|Frase)
    p_relevante_frase = p_frase_relevante*p_relevante
    
    #P(Irrelevante|Frase)
    p_irrelevante_frase = p_frase_irrelevante*p_irrelevante
    
    if p_relevante_frase > p_irrelevante_frase:
        return 1 #relevante
    else:
        return 0 #irrelevante
    
print(classificador_naive_bayes("tá"))
    
    

1


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [36]:
i=0

for tweet in tweets_teste["Teste"]:
    
    tweet_clean = cleanup(tweet)
    lista_tweet = tweet_clean.split()
    p_relevante_frase = 0
    p_irrelevante_frase = 0
    
    for k in lista_tweet:
        
        try:
        
            #P(Frase|Relevante)
            p_frase_relevante = freq_r.loc[k].prod()

            #P(Frase|Irrelevante)
            p_frase_irrelevante = freq_ir.loc[k].prod()

            #P(Relevante|Frase)
            p_relevante_frase = p_frase_relevante*p_relevante

            #P(Irrelevante|Frase)
            p_irrelevante_frase = p_frase_irrelevante*p_irrelevante
        
        except:
            
            p_relevante_frase += 1/(total + soma_r)
            p_irrelevante_frase += 1/(total + soma_ir)
            
            
    if p_relevante_frase > p_irrelevante_frase:
        tweets_teste["Treinamento"][i]=1
    
    else:
        tweets_teste["Treinamento"][i]=0
        
    i+=1
    

KeyError: 'Treinamento'

In [37]:
 tweets_teste.head(10)

,Teste,Classificação
0,eu gosto do conceito de 50% do uber eats que n...,0.0
1,o uber errando o caminho sendo q eu to atrasad...,1.0
2,mano minha cabeça eh meio doida tipo sair pra ...,0.0
3,"@daniela_moraess olá, @daniela_moraess! mandam...",0.0
4,"o uber louquissimo, dirigindo que nem piloto d...",1.0
5,rt @julialfeital: uber mulher é outra história...,1.0
6,incrível como isso siga minha energia vital ne...,0.0
7,o meu que ódio eu to atrasada e tive que me tr...,0.0
8,e quando você tá no ônibus e fica pensando que...,1.0
9,a difícil arte de pegar um uber segunda a noit...,1.0


In [38]:
Classificação=tweets_teste['Classificação']
Treinamento=tweets_teste['Treinamento']
pd.crosstab(Classificação, Treinamento)


KeyError: 'Treinamento'

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**